In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("synthetic_data.csv")
courses = list(df["course"].unique())
course_topics = list(df["course_topic"].unique())

In [4]:
course_encoder = LabelEncoder()
topic_encoder = LabelEncoder()

In [5]:
df["course"] = course_encoder.fit_transform(df["course"])
df["course_topic"] = topic_encoder.fit_transform(df["course_topic"])

In [6]:
scaler = MinMaxScaler()

In [7]:
df[
    [
        "course_grade",
        "easy_correct",
        "medium_correct",
        "hard_correct",
        "days_to_deadline",
    ]
] = scaler.fit_transform(
    df[
        [
            "course_grade",
            "easy_correct",
            "medium_correct",
            "hard_correct",
            "days_to_deadline",
        ]
    ].fillna(0)
)

In [8]:
df.head()

,user,course,course_grade,course_topic,easy_correct,medium_correct,hard_correct,upcoming_assignment,days_to_deadline,target
0,1,0,0.800459,60,0.912297,0.513660,0.439371,1,0.393914,0.553450
1,1,0,0.800459,37,0.774239,0.800331,0.447982,1,0.592694,0.478065
2,1,0,0.800459,95,0.269247,0.530319,0.652642,1,0.484431,0.621308
3,1,0,0.800459,101,0.746112,0.332602,0.917636,1,0.943728,0.670455
4,1,0,0.800459,51,0.989983,0.992933,0.398950,0,0.000000,0.342731


In [9]:
df.describe()

,user,course,course_grade,course_topic,easy_correct,medium_correct,hard_correct,upcoming_assignment,days_to_deadline,target
count,7300.000000,7300.000000,7300.000000,7300.000000,7300.000000,7300.000000,7300.000000,7300.00000,7300.000000,7300.000000
mean,25.500000,2.746575,0.472724,72.500000,0.480250,0.526988,0.551401,0.69137,0.350547,0.471289
std,14.431858,1.691397,0.259415,42.148468,0.269674,0.267965,0.264979,0.46196,0.335135,0.149529
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,13.000000,1.000000,0.258115,36.000000,0.253689,0.315366,0.346933,0.00000,0.000000,0.364784
50%,25.500000,3.000000,0.457056,72.500000,0.474373,0.536311,0.569196,1.00000,0.288848,0.483302
75%,38.000000,4.000000,0.695063,109.000000,0.700770,0.752885,0.768345,1.00000,0.646978,0.581450
max,50.000000,5.000000,1.000000,145.000000,1.000000,1.000000,1.000000,1.00000,1.000000,0.919125


In [11]:
train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [12]:
class StudyDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        course = row['course']
        topic = row['course_topic']
        grade = row['course_grade']
        easy_correct = row['easy_correct']
        medium_correct = row['medium_correct']
        hard_correct = row['hard_correct']
        upcoming_assignment = row['upcoming_assignment']
        days_to_deadline = row['days_to_deadline']
        target = row['target']

        inputs = {
            'course': torch.tensor(course, dtype=torch.long),
            'topic': torch.tensor(topic, dtype=torch.long),
            'features': torch.tensor([grade, easy_correct, medium_correct, hard_correct, upcoming_assignment, days_to_deadline], dtype=torch.float)
        }

        return inputs, torch.tensor(target, dtype=torch.float)

In [24]:
class StudyRecommenderModel(nn.Module):
    def __init__(self, n_courses, n_topics, emb_dim, hidden_size):
        super(StudyRecommenderModel, self).__init__()

        self.course_embedding = nn.Embedding(n_courses, emb_dim)
        self.topic_embedding = nn.Embedding(n_topics, emb_dim)

        self.fc1 = nn.Linear(emb_dim * 2 + 6, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, course, topic, features):
        course_emb = self.course_embedding(course)
        topic_emb = self.topic_embedding(topic)

        x = torch.cat([course_emb, topic_emb, features], dim=1)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))

        return x

In [28]:
model = StudyRecommenderModel(len(courses), len(course_topics), 32, 64)
crit = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=0.0001)

train_dataset = StudyDataset(train_df)
val_dataset = StudyDataset(val_df)
test_dataset = StudyDataset(test_df)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

epochs = 50

In [29]:
print(len(courses), len(course_topics))

6 146


In [30]:
for epoch in range(epochs):
    model.train() 
    train_loss = 0
    
    for inputs, target in train_loader:
        course = inputs['course']
        topic = inputs['topic']
        features = inputs['features']
        output = model(course, topic, features)
        loss = crit(output.squeeze(), target)
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    
    model.eval() 
    val_loss = 0
    
    with torch.no_grad():
        for inputs, target in val_loader:
            course = inputs['course']
            topic = inputs['topic']
            features = inputs['features']
            
            output = model(course, topic, features)
            loss = crit(output.squeeze(), target)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

Epoch 1/50, Train Loss: 0.0237, Val Loss: 0.0214
Epoch 2/50, Train Loss: 0.0226, Val Loss: 0.0210
Epoch 3/50, Train Loss: 0.0221, Val Loss: 0.0205
Epoch 4/50, Train Loss: 0.0216, Val Loss: 0.0200
Epoch 5/50, Train Loss: 0.0209, Val Loss: 0.0195
Epoch 6/50, Train Loss: 0.0200, Val Loss: 0.0187
Epoch 7/50, Train Loss: 0.0188, Val Loss: 0.0176
Epoch 8/50, Train Loss: 0.0173, Val Loss: 0.0163
Epoch 9/50, Train Loss: 0.0158, Val Loss: 0.0153
Epoch 10/50, Train Loss: 0.0145, Val Loss: 0.0143
Epoch 11/50, Train Loss: 0.0135, Val Loss: 0.0137
Epoch 12/50, Train Loss: 0.0129, Val Loss: 0.0134
Epoch 13/50, Train Loss: 0.0125, Val Loss: 0.0129
Epoch 14/50, Train Loss: 0.0121, Val Loss: 0.0127
Epoch 15/50, Train Loss: 0.0118, Val Loss: 0.0125
Epoch 16/50, Train Loss: 0.0116, Val Loss: 0.0122
Epoch 17/50, Train Loss: 0.0114, Val Loss: 0.0120
Epoch 18/50, Train Loss: 0.0112, Val Loss: 0.0118
Epoch 19/50, Train Loss: 0.0110, Val Loss: 0.0117
Epoch 20/50, Train Loss: 0.0108, Val Loss: 0.0116
Epoch 21/

In [31]:
torch.save(model.state_dict(), 'model.pth')

In [32]:
model.eval() 
test_loss = 0

with torch.no_grad():
    for inputs, target in test_loader:
        course = inputs['course']
        topic = inputs['topic']
        features = inputs['features']
        
        # Forward pass
        output = model(course, topic, features)
        loss = crit(output.squeeze(), target)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

Test Loss: 0.0108


In [45]:
#run inference
model.eval()
inputs, target = next(iter(test_loader))
course = inputs['course']
topic = inputs['topic']
features = inputs['features']
output = model(course, topic, features)
#print predicted target values
print(output.squeeze())
#print actual target values
print(target)
print(inputs)

tensor([0.3040, 0.3680, 0.5606, 0.3449, 0.6055, 0.4671, 0.2949, 0.4951, 0.5772,
        0.4936, 0.3227, 0.6125, 0.3031, 0.5943, 0.2852, 0.5650, 0.3116, 0.6553,
        0.4854, 0.3120, 0.6191, 0.5063, 0.4899, 0.5140, 0.4590, 0.6758, 0.4095,
        0.5859, 0.5589, 0.4904, 0.5676, 0.5849], grad_fn=<SqueezeBackward0>)
tensor([0.3191, 0.2362, 0.4871, 0.3227, 0.6152, 0.7082, 0.1313, 0.6574, 0.5998,
        0.5968, 0.3232, 0.3853, 0.3543, 0.4876, 0.2995, 0.6469, 0.1298, 0.5800,
        0.5646, 0.3325, 0.6215, 0.5415, 0.3869, 0.5793, 0.4324, 0.5151, 0.3469,
        0.6622, 0.5699, 0.4767, 0.5617, 0.7295])
{'course': tensor([0, 1, 4, 5, 3, 1, 1, 3, 5, 4, 1, 4, 0, 2, 5, 5, 4, 3, 4, 5, 3, 0, 4, 1,
        5, 4, 1, 1, 3, 5, 1, 4]), 'topic': tensor([101,  88,  99,  86, 117,  59,  24, 132,   1, 123,   5, 128,  37, 113,
        100,  75,   3,  91,   3,  65, 117,  95,  84, 106,  70,  17, 103, 103,
        130,  48,  87,  34]), 'features': tensor([[0.5217, 0.4947, 0.7334, 0.8539, 0.0000, 0.0000],
    

In [48]:
import joblib

joblib.dump(course_encoder, 'course_encoder.pkl')
joblib.dump(topic_encoder, 'topic_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']